In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Compact Layer 1

In [2]:
def CompactLayer1(layers, kernel_sizes, strides, paddings, biases, Relus):
    '''
    Args:
        layers       (list) : List of tuples consisting of number of in channels and out channels
        kernel_sizes (list) : List of tuples consisting of kernel sizes
        strides      (list) : List of tuples consisting of stride length. Can be None or tuple
        paddings     (list) : List of tuples consisting of padding dim. Can be None or tuple
        biases       (list) : List of boolean value stating whether bias is required or not.
        Relus        (list) : List of boolean value stating whether ReLU 
                              layer is required or not.True or False
    
    Returns:
        network  (nn.Sequential) : Compact layer consisting of sequence of Convolution layer and ReLUs.
        
    Example:
    >>> layers = [(3,64),(64,64)]
    >>> kernel_sizes = [(3,3),(5,5)]
    >>> strides = [(1,1),None]
    >>> paddings = [None,(1,1)]
    >>> biases = [None, True]
    >>> Relus = [True,False]
    >>> net = CompactLayer1(layers, kernel_sizes, strides, paddings, biases, Relus)
    >>> net
    Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
      (2): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    )

    '''
    
    model = []
    
    # The length of all lists must be same
    assert len(layers) == len(kernel_sizes) == len(strides) ==\
            len(paddings) == len(biases) == len(Relus)
    
    for i in range(len(layers)):
        
        stride = strides[i]
        padding = paddings[i]
        bias = biases[i]
        
        if stride is None:
            # If the stride is None the default value 1 is set
            stride = 1
        if padding is None:
            # If the padding is None the default value 0 is set
            padding = 0
        if bias is None:
            # If bias is None it is set True
            bias = True
        
        conv2d = nn.Conv2d(layers[i][0], layers[i][1], kernel_size=kernel_sizes[i], stride=stride, 
                           padding=padding, bias=bias)
        
        model.append(conv2d)
        
        if Relus[i]:
            model.append(nn.ReLU())
    
    network = nn.Sequential(*model)
    return network

In [10]:
layers = [(3,64), (64,64)]
kernel_sizes = [(3,3), (5,5)]
strides = [(1,1), None]
paddings = [None, (1,1)]
biases = [None, True]
Relus = [True, False]
net = CompactLayer1(layers, kernel_sizes, strides, paddings, biases, Relus)

In [11]:
net

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
)

In [16]:
# The convolutional layers change the number of channels from 3 to 64
channel = torch.ones((1, 3, 28, 28))
net(channel).size()

torch.Size([1, 64, 24, 24])

# Compact Layer 2

In [3]:
def CompactLayer2(features, biases, Relus, dropouts):
    '''
    Args:
        features     (list) : List of tuples consisting of in features and 
                              out features for the Linear layer
        biases       (list) : List of boolean value stating whether bias is required or not.
        Relus        (list) : List of boolean value stating whether ReLU 
                              layer is required or not.True or False
        dropouts     (list) : List of tuples consisting whether dropout 
                              is required or not and the dropout parameter p.
    
    Returns:
        network  (nn.Sequential) : Compact layer consisting of sequence of 
                                   Linear layers, ReLUs and Dropout Layers.
    
    Example:
    >>> features = [(3,3),(3,5)]
    >>> biases = [True, None]
    >>> Relus = [True, True]
    >>> dropouts = [(True, None), (False, 0.5)]
    >>> network = CompactLayer2(features, biases, Relus, dropouts)
    >>> network
    Sequential(
      (0): Linear(in_features=3, out_features=3, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.5)
      (3): Linear(in_features=3, out_features=5, bias=True)
      (4): ReLU()
    )
    '''
    
    model = []
    
    # The length of all lists must be same
    assert len(features) == len(dropouts) == len(biases) == len(Relus)
    
    for i in range(len(features)):
        
        in_features = features[i][0]
        out_features = features[i][1]
        is_drop, dropout = dropouts[i]
        bias = biases[i]
        
        if dropout is None:
            # If the dropout probablity is None the default value 1 is set
            dropout = 0.5
        if bias is None:
            # If bias is None it is set True
            bias = True
        
        linear_layer = nn.Linear(in_features=in_features, out_features=out_features, bias=bias)
        
        model.append(linear_layer)
        
        if Relus[i]:
            model.append(nn.ReLU())
        
        if is_drop:
            model.append(nn.Dropout(p=dropout))
            
    network = nn.Sequential(*model)
    return network

In [4]:
features = [(3,3),(3,5)]
biases = [True, None]
Relus = [True, True]
dropouts = [(True, None), (False, 0.5)]
network = CompactLayer2(features, biases, Relus, dropouts)

In [5]:
network

Sequential(
  (0): Linear(in_features=3, out_features=3, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.5)
  (3): Linear(in_features=3, out_features=5, bias=True)
  (4): ReLU()
)

In [6]:
# linear is a Float Tensor of size 3 
# The architecture is Linear (3, 3) -> ReLU -> Dropout(p=0.5) -> Linear (3, 5) -> ReLU
# Output will be Float Tensor of size 5
linear = torch.FloatTensor([[1, 2, 3]])
network(linear)